## Analysis

In [1]:
import pickle

In [ ]:
distances = pd.read_csv('data/OD_Pairs_Distances.csv')
population = pd.read_excel('data/BG_master.xlsx')
bg_included = pd.read_csv('data/bg_pop_included.csv')

In [ ]:
dist_miles = distances.pivot(index='block_group', columns='pod_id', values='Miles')

In [ ]:
dist = dist_miles.values
N = population['population'].values
prop = bg_included['all_included'].values

In [4]:
infile = open('pods_open_total','rb')
pods_total = pickle.load(infile)
infile.close()

In [ ]:
infile = open('pods_open_minmax','rb')
pods_minmax = pickle.load(infile)
infile.close()

In [6]:
pods_total.keys()

dict_keys([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46])

### Comparing average and maximum distance by PODs opened

### Histograms

For each element of the dictionary

In [ ]:
dist_min_total = [N[j] * [dist[j,i]] for [j,i] in block_pod_list]
dist_min_total_flat = [item for sublist in dist_min_total for item in sublist]

### POD capacity

In [ ]:
capacity_pod_all = [[N[j], i] for [j,i] in block_pod_list]

In [ ]:
capacity_pod = {}
for [N, i] in capacity_pod_all:
    if i not in capacity_pod:
        capacity_pod[i] = N
    else:
        capacity_pod[i] += N

In [ ]:
max(capacity_pod.values())

In [ ]:
min(capacity_pod.values())